Created Database, Added appehelion and perihelion distances, and Class of NEOs

In [9]:
import re
import requests
import pathlib
import sqlite3

# defining the directory and name of the NEODyS data
raw_data_dir = pathlib.Path("raw_data/")
raw_data_file = pathlib.Path("neodys.cat")

raw_data_filepath = raw_data_dir / raw_data_file
print(f"Our NEODyS file path: {raw_data_filepath}")

pathlib.Path.mkdir(raw_data_dir, exist_ok=True)

Our NEODyS file path: raw_data/neodys.cat


In [10]:
# Download the NEODyS file and store it locally but updates if the website updates
response = requests.get("https://newton.spacedys.com/~neodys2/neodys.cat")
download_file_path = pathlib.Path(raw_data_filepath)
with download_file_path.open(mode="wb+") as file_obj:
    file_obj.write(response.content)

In [11]:
# set the placeholder dict where the data will be stored, Data cleaning
neo_dict = []

# Open the NEODys file. Ignore the header (first 6 rows)
with open(raw_data_filepath) as f_temp:
    neo_data = f_temp.readlines()[6:]


    for neo_data_line_f in neo_data:
        neo_data_line = neo_data_line_f.split()
        neo_dict.append(
            {
                "Name": neo_data_line[0].replace("'", ""),
                "Epoch_MJD": float(neo_data_line[1]),
                "SemMajAxis_AU": float(neo_data_line[2]),
                "Ecc_": float(neo_data_line[3]),
                "Incl_deg": float(neo_data_line[4]),
                "LongAscNode_deg": float(neo_data_line[5]),
                "ArgP_deg": float(neo_data_line[6]),
                "MeanAnom_deg": float(neo_data_line[7]),
                "AbsMag_": float(neo_data_line[8]),
                "SlopeParamG_": float(neo_data_line[9]),
            }
        )



In [12]:
neo_dict[:1]

[{'Name': '433',
  'Epoch_MJD': 60000.0,
  'SemMajAxis_AU': 1.458129137561811,
  'Ecc_': 0.22278187193561982,
  'Incl_deg': 10.827822059970396,
  'LongAscNode_deg': 304.2870240036129,
  'ArgP_deg': 178.92698564290663,
  'MeanAnom_deg': 110.77766380976027,
  'AbsMag_': 10.91,
  'SlopeParamG_': 0.46}]

In [13]:
# Now creation of the database for NEODyS
database_dir = pathlib.Path("../databases/neos/")
database_file = pathlib.Path("neodys.db")
database_filepath = database_dir / database_file

# create directory
pathlib.Path.mkdir(database_dir, parents=True, exist_ok=True)

# Establish a connection to the database and set a cursor
neodys_db_con = sqlite3.connect(database_filepath)
neodys_db_cur = neodys_db_con.cursor()

In [14]:
# inserting raw data into database
neodys_db_cur.executemany(
    "INSERT OR IGNORE INTO main"
    "(Name,"
    "Epoch_MJD, "
    "SemMajAxis_AU, "
    "Ecc_, "
    "Incl_deg, "
    "LongAscNode_deg, "
    "ArgP_deg, "
    "MeanAnom_deg, "
    "AbsMag_, "
    "SlopeParamG_) "

    "VALUES (:Name, "
    ":Epoch_MJD, "
    ":SemMajAxis_AU, "
    ":Ecc_, "
    ":Incl_deg, "
    ":LongAscNode_deg, "
    ":ArgP_deg, "
    ":MeanAnom_deg, "
    ":AbsMag_, "
    ":SlopeParamG_)",
    neo_dict,
)

neodys_db_con.commit()

In [17]:
for col_name  in ["Aphel_AU", "Perihel_AU"]:
    sql_col_create = f"ALTER TABLE main ADD COLUMN {col_name} FLOAT"

    try:
        neodys_db_cur.execute(sql_col_create)
        neodys_db_con.commit()
    except sqlite3.OperationalError:

        pass

In [18]:
# fetching the data to make sure it has been transported and executes
neodys_db_cur.execute("SELECT Name, SemMajAxis_AU, Ecc_ FROM main LIMIT 5")
_neo_data = neodys_db_cur.fetchall() # outputs list of tuples
print(_neo_data)

[('433', 1.458129137561811, 0.22278187193561982), ('719', 2.636157148592985, 0.5467795300870554), ('887', 2.4725249968381324, 0.571093307295404), ('1036', 2.665298762177105, 0.5328259144094559), ('1221', 1.9201538890303416, 0.4347182162486856)]


In [19]:
neodys_db_cur.execute("SELECT Name, SemMajAxis_AU, Ecc_ FROM main")

# Fetching cursors
_neo_data = neodys_db_cur.fetchall()

_neo_deriv_param_dict = []
for _neo_data_line_f in _neo_data:
    _neo_deriv_param_dict.append(
        {
            "Name": _neo_data_line_f[0],
            "Aphel_AU": (1.0 + _neo_data_line_f[2]) * _neo_data_line_f[1],
            "Perihel_AU": (1.0 - _neo_data_line_f[2]) * _neo_data_line_f[1],
        }
    )

neodys_db_cur.executemany(
    "UPDATE main SET Aphel_AU = :Aphel_AU, Perihel_AU = :Perihel_AU "
    "WHERE Name = :Name",
    _neo_deriv_param_dict,
)

neodys_db_con.commit()

In [20]:
def neo_class(sem_maj_axis_au: float,
            peri_helio_au: float,
            ap_helio_au: float) -> str:

    """
    Classifies Near-Earth Objects (NEOs) based on their orbital parameters.

    Parameters
    ----------
    sem_maj_axis_au : float
        Semi-major axis of the NEO's orbit in astronomical units (AU).
    peri_helio_au : float
        Perihelion distance of the NEO's orbit in astronomical units (AU).
    ap_helio_au : float
        Aphelion distance of the NEO's orbit in astronomical units (AU).

    Returns
    -------
    str
        The classification of the NEO. Possible values are:
        - "Amor"
        - "Apollo"
        - "Aten"
        - "Atira"
        - "Other"

    Notes
    -----
    The classification is based on the criteria provided by NASA's
    Center for Near Earth Object Studies (CNEOS):
    https://cneos.jpl.nasa.gov/about/neo_groups.html

    Examples
    --------
    >>> neo_class(1.1, 1.1, 1.2)
    'Amor'
    >>> neo_class(1.1, 0.9, 1.2)
    'Apollo'
    >>> neo_class(0.9, 0.9, 1.0)
    'Aten'
    >>> neo_class(0.9, 0.9, 0.9)
    'Atira'
    >>> neo_class(1.5, 1.5, 1.5)
    'Other'
    """

    if (sem_maj_axis_au > 1.0) and (1.017 < peri_helio_au < 1.3):
        neo_type = "Amor"

    elif (sem_maj_axis_au > 1.0) and (peri_helio_au < 1.017):
        neo_type = "Apollo"

    elif (sem_maj_axis_au < 1.0) and (ap_helio_au > 0.983):
        neo_type = "Aten"

    elif (sem_maj_axis_au < 1.0) and (ap_helio_au < 0.983):
        neo_type = "Atira"

    else:
        neo_type = "Other"

    return neo_type

In [21]:
def create_col(sqlitecon: sqlite3.Connection,
                sqlitecur: sqlite3.Cursor,
                table: str,
                col_name: str,
                col_type: str) -> None:

    """
    Function to create new columns in table database.
    """

    sql_col_create = f"ALTER TABLE {table} ADD COLUMN {col_name} {col_type}"

    try:
        sqlitecur.execute(sql_col_create)
        sqlitecon.commit()
    except sqlite3.OperationalError:
        pass


In [22]:
create_col(sqlitecon=neodys_db_con,
            sqlitecur=neodys_db_cur,
            table="main",
            col_name="NEOClass",
            col_type="Text")

In [23]:
# get the orbital elements to compute the NEO class
neodys_db_cur.execute("SELECT Name, SemMajAxis_AU, Perihel_AU, Aphel_AU FROM main")

# fetch the data
neo_data = neodys_db_cur.fetchall()

# fetchall gives list of tuple, iterate and add it in the dict.abs
neo_class_param_dict = []
for neo_data_row in neo_data:
    neo_class_param_dict.append(
        {
            "Name": neo_data_row[0],
            "NEOClass": neo_class(
                sem_maj_axis_au=neo_data_row[1],
                peri_helio_au=neo_data_row[2],
                ap_helio_au=neo_data_row[3]
            )
        }
    )

# Insert into database
neodys_db_cur.executemany(
    "UPDATE main SET NEOClass = :NEOClass " # extra space not to concatenate WHERE statement
    "WHERE Name = :Name",
    neo_class_param_dict
)

neodys_db_con.commit()